<a href="https://colab.research.google.com/github/jiwoong2/deeplearning/blob/main/%EC%88%98%EC%B9%98%EB%AF%B8%EB%B6%84%EC%9D%84_%ED%86%B5%ED%95%9C_%ED%95%99%EC%8A%B5_%EA%B5%AC%ED%98%84.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pylab as plt

#Gradient, softmax, corss_entorpy_error, sigmoid, sigmoid_grad 구현

In [ ]:
def _numerical_gradient_no_batch(f, x): # 수치 gradient, 앞의 수치미분과 같은 개념
    h = 1e-4 # 0.0001
    grad = np.zeros_like(x) # x와 형상이 같은 배열을 생성(이 경우 x와 같은 차원의 제로 벡터)

    # gradient는 각 축에 대한 방향 미분을 모아놓은 벡터이므로 각 방향에 대한 수치미분을 grad 벡터에 차례로 대입하는 과정이다.
    for idx in range(x.size):
        tmp_val = x[idx]

        # f(x+h) 계산
        x[idx] = float(tmp_val) + h
        fxh1 = f(x)

        # f(x-h) 계산
        x[idx] = tmp_val - h
        fxh2 = f(x)

        grad[idx] =  (fxh1 - fxh2) / (2*h)
        x[idx] = tmp_val # 값 복원

    return grad

In [ ]:
def numerical_gradient(f, X): # 배치처리를 한 경우(행렬, 텐서 미분)
    if X.ndim == 1: # 1차원, 그러니까 행렬로 묶이지 않은 벡터인 경우
        return _numerical_gradient_no_batch(f, X)
    else:
        grad = np.zeros_like(X)

        for idx, x in enumerate(X): # enumerate 설명1, 행벡터를 차례로 불러와 미분한다.
            grad[idx] = _numerical_gradient_no_batch(f, x)

        return grad

In [ ]:
def cross_entropy_error(y, t):
    if y.ndim == 1: # y가 벡터인 경우, 배치처리를 안 한 경우
        t = t.reshape(1, t.size) # 10차원 벡터를 행렬로 변환.(개념상.)
        y = y.reshape(1, y.size)

    if t.size == y.size: # 라벨이 원-핫 인코딩이 돼어 있다면, 그러니까 입력 값이 10차원 벡터라면, size함수는 메트릭스의 원소갯수를 반환한다.
        t = t.argmax(axis=1) # 원-핫 인코딩 이전으로 되돌리기 위해 작성.

    batch_size = y.shape[0]
    return -np.sum(np.log(y[np.arange(batch_size), t])) / batch_size # 설명1, 확률벡터로 이루어진 행렬에서 알맞은 인덱스를 얻기위한 과정.

In [ ]:
def softmax(x):
    if x.ndim == 2:   # x가 행렬일 경우, 즉 배치처리를 해서 각 행이 확률벡터인 메트릭스를 입력받은 경우.
        x = x.T # 각 행이 확률벡터인 초기 메트릭스에서 각 열이 확률벡터인 메트릭스로 전치 시킨다.
        x = x - np.max(x, axis = 0) # np.max(x, axis = 0)는 각 열에 대한 최댓값을 저장한 리스트(벡터)를 반환 한다. 그리고 x에 대한 - 연산은 각 열의 확률벡터에 각 열의 최댓값을 빼주는 연산이다.(밑의 오버플로우 방지를 위한 최댓값 빼기와 같음.)
        y = np.exp(x) / np.sum(np.exp(x), axis = 0) # 위와 같은 행렬과 벡터의 연산 원리.( 밑의 예제 참고.)
        return y.T

    x = x - np.max(x) # 오버플로를 방지하기위해 작성. 노트참고. , vector에 scalar를 빼면 numpy에서 알아서 원소별로 연산을 한다.
    return np.exp(x) / np.sum(np.exp(x))  # normalize

In [ ]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))  # np.exp(x)는 밑이 자연상수 e 이고 x가 지수인 수를 반환한다.

In [ ]:
def sigmoid_grad(x):
    return (1.0 - sigmoid(x)) * sigmoid(x)

# Two layer net
입력층 - 은닉층 - 출력층 으로 이루이진 간단한 신경망 구조

In [ ]:
class TwoLayerNet:

    def __init__(self, input_size, hidden_size, output_size, weight_init_std = 0.01): # 파라미터들을 초기화한다. 편향은 0, 가중치는 지정한 정규분포에서 랜덤하게 초기화 한다.
                                                                                      # std는 표준편차이고 특별히 지정하지 않으면 0.01로 지정한다.
        self.params = {} # 빈 딕셔너리.

        # 입력층에서 은닉층으로의 아파인 변환에 필요한 가중치와 편향
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size) # 표준정규분포(평균이 0이고 표준편차가 1)에 상수 a를 곱하면 평균이 0이고 표준편차가 a인 정규분포를 생성한다.
        self.params['b1'] = np.zeros(hidden_size) # bias는 0으로 지정한다.

        # 은닉층에서 출력층으로의 아파인 변환에 필요한 가중치와 편향
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size)
        self.params['b2'] = np.zeros(output_size)

    def predict(self, x):
        W1, W2 = self.params['W1'], self.params['W2']
        b1, b2 = self.params['b1'], self.params['b2']

        # affain 변환
        a1 = np.dot(x, W1) + b1 # 결과적으로 50차원 벡터가 반환 한다.(net2의 경우)
        z1 = sigmoid(a1) # 활성화 함수
        a2 = np.dot(z1, W2) + b2 # 결과적으로 10차원 벡터가 반환 한다.(net2의 경우)
        y = softmax(a2) # a2를 확률벡터로 변환한다.

        return y

    def loss(self, x, t): # 추론한 확률분포와 라벨의 확률분호의 거리를 측정 한다.(cross entropy)
        y = self.predict(x)

        return cross_entropy_error(y, t)

    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis = 1) # 행을 따라 최댓값의 인덱스를 리스트로 반환한다.
        t = np.argmax(t, axis = 1)

        accuracy = np.sum(y == t) / float(x.shape[0]) # 정답의 수를 입력한 데이터 수로 나눈다.(배치 처리)

        return accuracy

    def numerical_gradient(self, x, t):
        loss_W = lambda W: self.loss(x, t)

        grads = {} # 빈 딕셔너리
        grads['W1'] = numerical_gradient(loss_W, self.params['W1']) # 785*50 행렬 반환 (gardient의 shape은 입력 shpae과 동일한다.)
        grads['b1'] = numerical_gradient(loss_W, self.params['b1']) # 50차원 백터 반환
        grads['W2'] = numerical_gradient(loss_W, self.params['W2']) # 50*10 행렬 반환
        grads['b2'] = numerical_gradient(loss_W, self.params['b2']) # 10차원 벡터 반환

        return grads

    def gradient(self, x, t): # 역전파를 구현한 코드. 수치미분은 계산비용이 너무 크기 때문 모델을 학습시키기에는 부적합하다.
        W1, W2 = self.params['W1'], self.params['W2']
        b1, b2 = self.params['b1'], self.params['b2']
        grads = {}

        batch_num = x.shape[0]

        # forward
        a1 = np.dot(x, W1) + b1
        z1 = sigmoid(a1)
        a2 = np.dot(z1, W2) + b2
        y = softmax(a2)

        #backward
        dy = (y - t) / batch_num
        grads['W2'] = np.dot(z1.T, dy)
        grads['b2'] = np.sum(dy, axis = 0)

        da1 = np.dot(dy, W2.T)
        dz1 = sigmoid_grad(a1) * da1
        grads['W1'] = np.dot(x.T, dz1)
        grads['b1'] = np.sum(dz1, axis = 0)

        return grads

# 간단한 예제 3-3-3

In [ ]:
net1 = TwoLayerNet(3, 3, 3) # 입력층 뉴련 3개 ,은닉층 뉴런 3개, 아웃풋 뉴런 3개인 구조를 가진 인스턴스를 생성한다.

In [ ]:
net1.params

In [ ]:
net1.predict(np.array([1,2,3])) # 3차원 확률벡털을 반환한다.

# MNIST 데이터셋에 학습하기.


In [ ]:
import tensorflow as tf

MNIST dataset 다운로드

In [ ]:
Mnist = tf.keras.datasets.mnist
(x_train, t_train), (x_test, t_test) = Mnist.load_data()

dataset 전처리

In [ ]:
# 이미지 데이터 전처리
x_train, x_test = x_train / 255.0, x_test / 255.0 # normalize -> 많은 경우 피쳐가 여러개이므로 큰 숫자를 가지는 피쳐의 영향력이 너무 커지므로 모든 피쳐값을 0부터 1까지의 값으로 정규화 한다.
x_train, x_test = x_train.reshape(60000, 784), x_test.reshape(10000, 784) # flatten
x_test = x_test.reshape(10000, 784)

# 라벨 one-hot encoding
num_category = 10
t_train = tf.keras.utils.to_categorical(t_train, num_category)
t_test = tf.keras.utils.to_categorical(t_test, num_category)
print(t_train[0])

In [ ]:
print(x_train.shape)
print(x_test.shape)

모델 설정

In [ ]:
network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10) # 784-50-10 구조의 인스턴스 생성

하이퍼파라미터 설정

In [ ]:
iters_num = 10000 # 반복 횟수
train_size = x_train.shape[0]
batch_size = 100 # 미니배치 크기
learning_rate = 0.1

train_loss_list = [] # 학습과정의 loss 변화값을 저장
train_acc_list = [] # 학습과정의 accuracy 변화 저장
test_acc_list = []

모델 학습

In [ ]:
iter_per_epoch = max(train_size / batch_size, 1) # epoch은 데이터를 다 소진할때까지 학습해야할 횟수를 뜻한다. train data의 경우 600번.

for i in range(iters_num):

    # 미니배치 획득
    batch_mask = np.random.choice(train_size, batch_size) # np.random.choice(5, 3) 의경우 0,1,2,3,4 에서 3가지 수를 랜던하게 추출한다.
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]

    # 기울기 계산
    grad = network.gradient(x_batch, t_batch) # 손실함수를 각 가중치와 편향으로 미분한 딕셔너리를 반환 받는다. {w1 : L/w1, ...}

    # 매개변수 갱신
    for key in ('W1', 'b1', 'W2', 'b2'):
        network.params[key] -= learning_rate * grad[key] # gradient decent를 이용한 매개변수 갱신

    # 학습 경과 기록
    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)

    # 1에폭당 정확도 계산
    if i % iter_per_epoch == 0:
        train_acc = network.accuracy(x_train, t_train)
        test_acc = network.accuracy(x_test, t_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        print("train acc, test acc |" + str(train_acc) + "," + str(test_acc))

그래프 그리기

In [ ]:
markers = {'train': 'o', 'test' : 's'}
x = np.arange(len(train_acc_list))
plt.plot(x, train_acc_list, label='train acc')
plt.plot(x, test_acc_list, label='test acc', linestyle='--')
plt.xlabel("epochs")
plt.ylabel("accuracy")
plt.ylim(0, 1.0)
plt.legend(loc = 'lower right')
plt.show()

# Learning rate를 늘려가며 그래프를 그리다보면 점점 성능이 좋아지다 어느순간 학습을 해도 정확도가 개선되지 않는다.